In [ ]:
import numpy as np
from scipy.io import loadmat
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import glob

# === Function to summarize each feature matrix ===
def summarize_feature(feat):
    return np.hstack([
        np.mean(feat, axis=1),
        np.std(feat, axis=1),
        np.min(feat, axis=1),
        np.max(feat, axis=1)
    ])

# === Load features from MATLAB .mat files ===
X = []
y = []

# All .mat files assumed to be in same folder
feature_files = sorted(glob.glob('features_sample*.mat'))

for f in feature_files:
    data = loadmat(f)
    mfcc = data['mfcc']
    centroid = data['centroid']
    rolloff = data['rolloff']
    mel = data['mel']
    chroma = data['chroma']

    # Combine summarized features into one vector
    feature_vector = np.hstack([
        summarize_feature(mfcc),
        summarize_feature(centroid),
        summarize_feature(rolloff),
        summarize_feature(mel),
        summarize_feature(chroma)
    ])
    X.append(feature_vector)

# Load labels from labels.txt
with open('labels.txt', 'r') as f:
    for line in f:
        y.append(line.strip())

X = np.array(X)
y = np.array(y)

# === Encode string labels into numeric values ===
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# === Train-test split (80% training, 20% testing) ===
x_train, x_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=1, stratify=y_encoded
)

# === Create and train SVM model with RBF kernel ===
classifier = SVC(kernel='rbf', random_state=1, C=1, gamma='auto')
classifier.fit(x_train, y_train)

# === Predict on test set ===
y_pred = classifier.predict(x_test)

# === Confusion matrix and accuracy calculation ===
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

accuracy = float(cm.diagonal().sum()) / len(y_test)
print(f"Model accuracy is: {accuracy * 100:.2f}%")
